In [64]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq groq==0.13.0 --progress-bar off
!pip install -qqq python-dotenv==1.0.1 --progress-bar off
!pip install groq
!pip install nltk

ERROR: To modify pip, please run the following command:
C:\Users\anavn\AppData\Local\Programs\Python\Python312\python.exe -m pip install -Uqqq pip --progress-bar off


^C


In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anavn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def chunk_transcript(data: pd.DataFrame) -> pd.DataFrame:
    
    # Ensure 'Transkript' column has no NaN values
    data['Transkript'] = data['Transkript'].fillna("").astype(str)

    # Chunking logic
    chunked_data = []
    current_speaker = None
    current_text = ""

    for index, row in data.iterrows():
        speaker = row['Sprecher']
        transcript = row['Transkript']

        if speaker != current_speaker:
            # Save previous chunk if exists
            if current_speaker is not None:
                chunked_data.append({
                    'Speaker': current_speaker,
                    'Transcript': current_text.strip()
                })
            
            # Start a new chunk
            current_speaker = speaker
            current_text = transcript
        else:
            # Continue appending to the same speaker's chunk
            current_text += " " + transcript if isinstance(transcript, str) else ""

    # Save the last chunk
    if current_speaker is not None:
        chunked_data.append({
            'Speaker': current_speaker,
            'Transcript': current_text.strip()
        })

    # Convert to DataFrame
    chunked_df = pd.DataFrame(chunked_data)
    
    return chunked_df


In [ ]:
def chunk_by_sentence(chunked_df: pd.DataFrame, min_tokens=256, max_tokens=512) -> pd.DataFrame:
    """
    Further splits chunks by sentence while ensuring each chunk is within a token range.

    Args:
        chunked_df (pd.DataFrame): Input DataFrame with 'Speaker' and 'Transcript' columns.
        min_tokens (int): Minimum number of tokens per chunk.
        max_tokens (int): Maximum number of tokens per chunk.

    Returns:
        pd.DataFrame: DataFrame with sentence-based chunked transcripts.
    """

    # Function to count tokens (approximate, assuming 1 word ≈ 1.2 tokens)
    def count_tokens(text):
        return len(text.split()) * 1.2  # Rough estimate

    # Initialize list for final merged chunks
    merged_chunks = []
    temp_chunk = []
    temp_token_count = 0
    speaker = None

    # Process each row
    for _, row in chunked_df.iterrows():
        sentence = row['Transcript']
        sentence_tokens = count_tokens(sentence)

        # If adding this chunk keeps us within MAX_TOKENS
        if temp_token_count + sentence_tokens <= max_tokens:
            if not temp_chunk:
                speaker = row['Speaker']  # Store speaker only for new chunks
            temp_chunk.append(sentence)
            temp_token_count += sentence_tokens
        else:
            # Save the previous chunk before starting a new one
            if temp_chunk:
                merged_chunks.append({
                    'Speaker': speaker,
                    'Transcript': " ".join(temp_chunk)
                })

            # Start a new chunk with the current sentence
            temp_chunk = [sentence]
            temp_token_count = sentence_tokens
            speaker = row['Speaker']

    # Save last chunk if any content remains
    if temp_chunk:
        merged_chunks.append({
            'Speaker': speaker,
            'Transcript': " ".join(temp_chunk)
        })

    # Convert to DataFrame
    final_merged_df = pd.DataFrame(merged_chunks)
    
    return final_merged_df

In [109]:
from groq import Groq
import pandas as pd 


# api_key = "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"
# api_key = "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"
# api_key = "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"
# api_key = "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"
api_key = "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"
# Initialize the Groq client
client = Groq(api_key=api_key)

print("Groq client initialized successfully!")

Groq client initialized successfully!


In [110]:
def extract_metadata(client, model_name: str, chunks: pd.DataFrame) -> pd.DataFrame:
    """
    Extract metadata from multiple chunks of a German transcript using the specified model.
    
    Args:
        client: The Groq client object.
        model_name: The name of the model to use (e.g., 'llama3-8b-8192').
        chunks: A DataFrame with 'Speaker' and 'Transcript' columns.
    
    Returns:
        A DataFrame containing the extracted metadata for all chunks.
    """
    all_metadata = []
    num_chunks = len(chunks)

    for i, row in chunks.iterrows():  # Iterate over DataFrame rows
        speaker = row["Speaker"]
        transcript = row["Transcript"]

        print(f"Processing chunk {i+1}/{len(chunks)} for Speaker: {speaker}...")

        # Prompt for metadata extraction
        prompt = f"""
        Extrahieren Sie die folgenden Informationen aus dem Transkript und geben Sie die Antwort auf Deutsch ein. Wenn die Antwort nicht gefunden wurde, geben Sie sie als %%% zurück:

        ### Persönliche Informationen:
        NAME: Der vollständige Name der Person (Vor- und Nachname, falls verfügbar).
        JAHRGANG: Das Geburtsjahr der Person.
        ORT: Der Geburtsort der Person.
        GESCHLECHT: Das Geschlecht der Person (z.B. männlich, weiblich, divers).
        BERUF: Der aktuelle Beruf der Person.

        ### Informationen über Vater und Partner:
        VAT_JG: Geburtsjahr des Vaters.
        VAT_KONFESSION: Religion des Vaters.
        VAT_HERKUN: Herkunft des Vaters.
        VAT_SCHULE: Schulbildung des Vaters.
        VAT_AUSBIL: Ausbildung des Vaters.
        VAT_STAND: Beruflicher Status des Vaters.
        VAT_POLOR: Politische Orientierung des Vaters.

        Transkript:
        {transcript}

        Metadaten:
        NAME:
        JAHRGANG:
        ORT:
        GESCHLECHT:
        BERUF:
        VAT_JG:
        VAT_KONFESSION:
        VAT_HERKUN:
        VAT_SCHULE:
        VAT_AUSBIL:
        VAT_STAND:
        VAT_POLOR:

        Fügen Sie keine Präambel ein.
        """

        # Generate response using the Groq client
        response = client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )

        # Extract metadata from the response
        metadata = response.choices[0].message.content

        # Parse metadata into a dictionary
        extracted_metadata = {"Speaker": speaker}  # Store speaker info
        for line in metadata.split("\n"):
            if ":" in line:
                key, value = line.split(":", 1)
                extracted_metadata[key.strip()] = value.strip()

        # Remove "%%%%" values before storing
        extracted_metadata = {k: (v if v != "%%%" else "") for k, v in extracted_metadata.items()}

        if i < num_chunks - 1:  # If not the last iteration
            extracted_metadata = {k: v.strip() + "," if v else v.strip() for k, v in extracted_metadata.items()}

        # Append metadata for this chunk
        all_metadata.append(extracted_metadata)

    # Convert metadata list into a DataFrame
    return pd.DataFrame(all_metadata)


        # if i < num_chunks - 1:  # If not the last iteration
        #     for k, v in extracted_metadata.items():
        #         if v:  # If the value is not empty
        #             existing_values = set([extracted_metadata[k]][0].strip().split(" "))  # Convert existing values to a set
        #             # existing_values.add(v)  # Add new value (set avoids duplicates)
        #             extracted_metadata[k] = " | ".join(existing_values)  # Join back without duplicates



        # # Append metadata for this chunk
        # all_metadata.append(extracted_metadata)


    #     # Check for duplicates column-wise
    #     is_duplicate = False
    #     for prev_metadata in all_metadata:
    #         for key in extracted_metadata.keys():
    #             if key in prev_metadata and extracted_metadata[key] == prev_metadata[key]:  
    #                 is_duplicate = True
    #                 continue
    #         if is_duplicate:
    #             break  

    #     # If the entire metadata row is not a duplicate, append it
    #     if not is_duplicate:
    #         all_metadata.append(extracted_metadata)

    # Convert metadata list into a DataFrame
    # return pd.DataFrame(all_metadata)

        # Remove "%%%%" values before storing
        # extracted_metadata = {k: (v if v != "%%%" else "") for k, v in extracted_metadata.items()}

        # Check for duplicates column-wise
    #     cleaned_metadata = {"Speaker": speaker}  # Keep speaker info
    #     for key, value in extracted_metadata.items():
    #         if key == "Speaker":
    #             continue  # Skip speaker key check
    #         is_duplicate = any(value == prev_row.get(key, None) for prev_row in all_metadata)
            
    #         if not is_duplicate:  # Add only if value is unique
    #             cleaned_metadata[key] = value

    #         for key in cleaned_metadata.keys():
    #             values = [v for v in cleaned_metadata[key].split(" | ") if v]  # Remove empty values
    #             cleaned_metadata[key] = " | ".join(values) 


    #     # If there’s any new unique metadata, add it
    #     if len(cleaned_metadata) > 1:  
    #         all_metadata.append(cleaned_metadata)

    # # Convert metadata list into a DataFrame
    # return pd.DataFrame(all_metadata)

In [111]:
import os
import time

folder_path = "Transcripts ADG0001-10"
MODEL = "llama-3.3-70b-versatile"

# List to store metadata for all files
all_metadata = []
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) & filename.endswith(".csv"):
        print(f"\nProcessing file: {filename}")
        input_data = pd.read_csv(file_path, sep=None, engine='python')
        speaker_chunks_df = chunk_transcript(input_data)  # Stores speaker-based chunks
        final_chunks_df = chunk_by_sentence(speaker_chunks_df)
        # print(final_chunks_df)
        

        # Extract metadata for the chunks
        llama_70b_responses = extract_metadata(client,MODEL , final_chunks_df)

        # Ensure that the response DataFrame contains metadata columns
        if not llama_70b_responses.empty:
            # Merge chunk outputs into a single row 
            merged_metadata = llama_70b_responses.apply(lambda col: ' '.join(col.dropna().astype(str)))
            
            for column in merged_metadata.index:
                unique_values = set([value.strip() for value in merged_metadata[column].strip().split(",")])
                list_unique_values = list(filter(None, unique_values))
                merged_metadata[column] = " | ".join(list_unique_values)

            # Add filename for reference
            # merged_metadata["Filename"] = filename  

            # Append to list
            all_metadata.append(merged_metadata)
        else:
            print(f"No metadata extracted from {filename}")
         
        time.sleep(0.5)

# Convert list of metadata rows into a single DataFrame
final_metadata_df = pd.DataFrame(all_metadata)



Processing file: file 1.csv
Processing chunk 1/21 for Speaker: INT_AH...
Processing chunk 2/21 for Speaker: IP_FA...
Processing chunk 3/21 for Speaker: INT_AH...
Processing chunk 4/21 for Speaker: IP_FA...
Processing chunk 5/21 for Speaker: INT_AH...
Processing chunk 6/21 for Speaker: IP_FA...
Processing chunk 7/21 for Speaker: IP_FA...
Processing chunk 8/21 for Speaker: IP_FA...
Processing chunk 9/21 for Speaker: IP_FA...
Processing chunk 10/21 for Speaker: IP_FA...
Processing chunk 11/21 for Speaker: IP_FA...
Processing chunk 12/21 for Speaker: IP_FA...
Processing chunk 13/21 for Speaker: IP_FA...
Processing chunk 14/21 for Speaker: IP_FA...
Processing chunk 15/21 for Speaker: INT_AH...
Processing chunk 16/21 for Speaker: IP_FA...
Processing chunk 17/21 for Speaker: IP_FA...
Processing chunk 18/21 for Speaker: IP_FA...
Processing chunk 19/21 for Speaker: IP_FA...
Processing chunk 20/21 for Speaker: IP_FA...
Processing chunk 21/21 for Speaker: INT_AH...


In [112]:
final_metadata_df

,Speaker,NAME,JAHRGANG,ORT,GESCHLECHT,BERUF,VAT_JG,VAT_KONFESSION,VAT_HERKUN,VAT_SCHULE,VAT_AUSBIL,VAT_STAND,VAT_POLOR
0,INT_AH | IP_FA,,1925,Hemer im Sauerland | Gelsenkirchen-Buer,weiblich,Leitung der rollenden Küche | Büroarbeit,,,,,Überseekaufmann | Ingenieurstudium,Kreisorganisationsleiter bei der Arbeitsfront ...,


In [107]:
pd.DataFrame(all_metadata)

,Speaker,NAME,JAHRGANG,ORT,GESCHLECHT,BERUF,VAT_JG,VAT_KONFESSION,VAT_HERKUN,VAT_SCHULE,VAT_AUSBIL,VAT_STAND,VAT_POLOR
0,INT_AH | IP_FA,,1925,Hemer im Sauerland,weiblich,Büroarbeit,,,,,Überseekaufmann,Kreisorganisationsleiter bei der Arbeitsfront ...,


In [108]:
final_metadata_df['VAT_STAND'][0]

'Kreisorganisationsleiter bei der Arbeitsfront | Geschäftsführer | Fabrikant'

In [37]:
" | ".join([value for value in set(final_metadata_df['GESCHLECHT'].to_list()[0].strip().split(" ")) if value != "" ])

'weiblich'

In [ ]:
[value for value in set(final_metadata_df['GESCHLECHT'].to_list()[0].strip().split(" ")) if value]

['weiblich']

In [36]:
final_metadata_df.to_csv("metadata_results.csv", index=False)